In [ ]:
!pip install bleach

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install datasets

     |████████████████████████████████| 163kB 5.6MB/s 
     |████████████████████████████████| 17.7MB 339kB/s 
     |████████████████████████████████| 245kB 56.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
!pip install transformers==4.2.1

     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 890kB 43.4MB/s 
     |████████████████████████████████| 2.9MB 20.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ec719cdc6ec5ea08887d40e430f9994568d64a56b72cbc605484ab2876f6a5bd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 686kB 4.0MB/s 
     |████████████████████████████████| 276kB 15.4MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 829kB 15.6MB/s 
     |████████████████████████████████| 1.3MB 22.4MB/s 
     |████████████████████████████████| 143kB 33.8MB/s 
     |████████████████████████████████| 296kB 18.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=09e63f54006c4d95463ad996048c326ce145ce3907477080c603ef93f9b0ae2a
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=d0caa742b3cbfdba6d8a5df4d9a3a210150025ee50bb3edc48814e4daa58e1f9
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 

In [ ]:
import datasets
from datasets import list_datasets, load_dataset, list_metrics, load_metric
datasets_list = list_datasets()
import torch
import re
import bleach
from bs4 import BeautifulSoup

In [ ]:
dataset  = datasets.load_dataset('hyperpartisan_news_detection', 'bypublisher')

Dataset hyperpartisan_news_detection downloaded and prepared to /root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/60aa536d5067f21aacb9ab08b94548649fd241c1e3cf6bb643d0a4a1b20bcf25. Subsequent calls will reuse this data.


In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
        num_rows: 600000
    })
    validation: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
        num_rows: 600000
    })
})


### Details of the data set

In [ ]:
print("Size of train dataset: ", dataset['train'].shape)

Size of train dataset:  (600000, 6)


In [ ]:
dataset['train']

Dataset({
    features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
    num_rows: 600000
})

In [ ]:
position = 1

In [ ]:
print(" Example of text: ", dataset['train'][position]['text'])
print(" Length of text: ", len(dataset['train'][position]['text'].split(" ")))

 Example of text:  <p>A Florida university will honor Trayvon Martin with a posthumous Bachelor of Science Degree in Aviation at a May 13 commencement ceremony. The degree will be accepted by his parents, Tracy Martin and Sabrina Fulton.</p> 

<p>Seventeen-year-old Martin was killed in February of 2012 by former neighborhood watch captain George Zimmerman, who is Hispanic, in self-defense. After a long, racially-charged public debate and court trial, Zimmerman was completely exonerated in July of 2013.</p> 

<p>Florida Memorial University officially <a href="https://www.facebook.com/FLMemorialUniv/" type="external">announced</a>the posthumous degree via Facebook on Wednesday.</p> 

<p>"The University will confer upon TRAYVON MARTIN a posthumous degree in Aeronautical Science with a concentration in Flight Education, in honor of the steps he took during his young life toward becoming a pilot," says the Facebook post. "This particular Bachelor&#8217;s degree is designated for those study

In [ ]:
print(" Class of above text: ",  dataset['train'][position]['hyperpartisan'])

 Class of above text:  True


### Preprocess Data

In [ ]:
def clean_text(text, label):
    """ Clean the input text and remove special characters """
#     soup = BeautifulSoup(text)
    text = bleach.clean(text,strip=True)
    text = text.replace('<p>', '')
    text = text.replace('</p>', '')
    text = text.replace('\n', '')
    text = text.replace('&amp;#160;', '')
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    if str(label) == 'True':
        new_label = 1
    else:
        new_label = 0
    return text, new_label


def convert_to_features(example):
    # Tokenize contexts and questions (as pairs of inputs)
    text_, target_ = clean_text(example['text'], example['hyperpartisan'])
    encodings = tokenizer.encode_plus(text_, pad_to_max_length=True, max_length=2048,
                                           add_special_tokens=True,
                                            return_token_type_ids=False,
                                            return_attention_mask=True,
                                            padding='max_length', truncation=True,
                                           )
    targets = torch.tensor(target_,dtype=torch.long)
    

    encodings.update({'targets': targets,
                      'attention_mask': encodings['attention_mask']})
    return encodings

In [ ]:
from transformers import LongformerTokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

In [ ]:
## Take subset of data
train_size = 20000
val_size = 5000
import numpy as np
train_indices = np.random.randint(0, len(dataset['train']), train_size)
val_indices = np.random.randint(0, len(dataset['validation']), val_size)
train_dataset = dataset['train'].select(train_indices)
val_dataset = dataset['validation'].select(val_indices)

In [ ]:
train_dataset =  train_dataset.map(convert_to_features, load_from_cache_file=False)
val_dataset =  val_dataset.map(convert_to_features, load_from_cache_file=False)

columns = ['input_ids', 'attention_mask', 'targets']
train_dataset.set_format(type='torch', columns=columns)
val_dataset.set_format(type='torch', columns=columns)

In [ ]:
torch.save(train_dataset, 'train_data.pt')
torch.save(val_dataset, 'valid_data.pt')

In [ ]:
import torch
from transformers import LongformerTokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
train_dataset = torch.load('train_data.pt')
val_dataset = torch.load('valid_data.pt')

In [ ]:
print(len(train_dataset[10]['input_ids']))

2048


/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:851: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


In [ ]:
print(len(train_dataset[10]['input_ids']))

2048


In [ ]:
print(train_dataset[10])

{'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'input_ids': tensor([   0, 1620,   52,  ...,    1,    1,    1]), 'targets': tensor(1)}


In [ ]:
tokenizer.decode(train_dataset[10]['input_ids'])

'<s>As we prepare for oral arguments Friday before a federal appeals court in Washington, D.C. in our appeal of our lawsuit challenging ObamaCare, it\'s important to note that a growing number of Americans reject the pro-abortion, government-run, and costly health care law. In a new poll released by <a href="http://www.rasmussenreports.com/public_content/politics/current_events/healthcare/health_care_law">Rasmussen</a>, 56% of likely voters favor repeal of ObamaCare, while 52% say the health care law is bad for the country. A clear majority of voters have favored repeal of the law in every Rasmussen survey but one since the measure was passed by Congress in March of last year. As you may recall, our legal team is preparing for critical arguments before a three-judge panel of a federal appeals court in Washington on Friday. We\'re appealing a lower court decision dismissing our federal lawsuit which challenges the constitutionality of ObamaCare. We filed <a href="">our lawsuit in federa

## Longformer-4096 Fine Tuning

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from datasets import load_metric
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from transformers import (
    AdamW,
    LongformerTokenizer,
    LongformerForSequenceClassification,
    get_linear_schedule_with_warmup
)
import logging
import argparse
import time

In [ ]:
from pytorch_lightning.loggers.wandb import WandbLogger
import os
from pathlib import Path
from string import punctuation

In [ ]:
class LonformerFineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(LonformerFineTuner, self).__init__()
        self.hparams = hparams        
        self.model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", return_dict=True, num_labels=2)
        self.tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
        self.output_dir = Path(self.hparams.output_dir)
        self.step_count = 0
        
        if self.hparams.freeze_embeds:
            self.freeze_embeds()
        if self.hparams.freeze_encoder:
            self.freeze_params(self.model.get_encoder())
            assert_all_frozen(self.model.get_encoder())
            
            
        n_observations_per_split = {
            "train": self.hparams.n_train,
            "validation": self.hparams.n_val,
            "test": self.hparams.n_test,
        }
        self.n_obs = {k: v if v >= 0 else None for k, v in n_observations_per_split.items()}
        self.em_score_list = []
        
    def freeze_params(self, model):
        for par in model.parameters():
            par.requires_grad = False
            
    def freeze_embeds(self):
        try:
            self.freeze_params(self.model.model.shared)
            for d in [self.model.model.encoder, self.model.model.decoder]:
                freeze_params(d.embed_positions)
                freeze_params(d.embed_tokens)
        except AttributeError:
            self.freeze_params(self.model.shared)
            for d in [self.model.encoder, self.model.decoder]:
                self.freeze_params(d.embed_tokens)

    def lmap(self, f, x):
        return list(map(f, x))

    def is_logger(self):
        return self.trainer.global_rank <= 0
    
    def parse_score(self, result):
        return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
        
    def forward(self, input_ids, attention_mask, labels):
        return self.model(
                input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

    def _step(self, batch):
        labels = batch["targets"]

        outputs = self(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels,
        )

        loss = outputs[0]

        return loss
    
    def ids_to_clean_text(self, generated_ids):
        gen_text = self.tokenizer.batch_decode(
            generated_ids,
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=True
        )
        return self.lmap(str.strip, gen_text)
    
    def _generative_step(self, batch) :
        
        labels = batch["targets"]

        outputs = self(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels,
        )
        
        loss = outputs[0]
        logits = F.softmax(outputs[1], dim=1)
        preds = torch.argmax(logits, 1)
        preds = preds.cpu().data.numpy()
        targets = labels.cpu().data.numpy()
#         print("Target and Preds: ", targets, preds)
        
        t0 = time.time()
            
        gen_time = (time.time() - t0) / batch["input_ids"].shape[0]  
    
        loss = self._step(batch)
        base_metrics = {'val_loss': loss}
        base_metrics.update(gen_time=gen_time)
        
    
        em_score  = accuracy_score(targets, preds)
        
        self.em_score_list.append(em_score)
        
        em_score = torch.tensor(em_score,dtype=torch.float32)
    
        base_metrics.update(accuracy=em_score)
        
        return base_metrics

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}
  
    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        return self._generative_step(batch)
    
    def validation_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        
        if len(self.em_score_list) <= 2:
            average_em_score = sum(self.em_score_list) / len(self.em_score_list) 
            
        else:
            latest_em_score = self.em_score_list[:-2]
            average_em_score = sum(latest_em_score) / len(latest_em_score) 
        
        average_em_score = torch.tensor(average_em_score,dtype=torch.float32)
        tensorboard_logs.update(accuracy=average_em_score)
        
        self.target_gen= []
        self.prediction_gen=[]
        return {"avg_val_loss": avg_loss, 
                "accuracy" : average_em_score,
                "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]
  
    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx,
                       closure, on_tpu=False, using_native_amp=False, using_lbfgs=False):
        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
            optimizer.step(closure=closure)
        optimizer.zero_grad()
        self.lr_scheduler.step()
  
    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict
    
    def train_dataloader(self):   
        n_samples = self.n_obs['train']
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
        t_total = (
            (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            // self.hparams.gradient_accumulation_steps
            * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler

        return dataloader

    def val_dataloader(self):
        n_samples = self.n_obs['validation']
        return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)
    
    def test_dataloader(self):
        n_samples = self.n_obs['test']
        test_dataset = get_dataset(tokenizer=self.tokenizer, type_path="test", num_samples=n_samples, args=self.hparams)
        
        return DataLoader(test_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)
    
    def on_save_checkpoint(self, checkpoint):
        save_path = self.output_dir.joinpath(model_prefix)
        self.model.config.save_step = self.step_count
        self.model.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)

In [ ]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # Log and save results to file
            output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))

In [ ]:
model_name = "longformer"
token_len = 4096
model_prefix = f"{model_name}-{token_len}"

In [ ]:
args_dict = dict(
    output_dir="", # path to save the checkpoints
    max_input_length=token_len,
    freeze_encoder=False,
    freeze_embeds=False,
    learning_rate=2e-5,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=1,
    eval_batch_size=1,
    num_train_epochs=25,
    gradient_accumulation_steps=1,
    n_gpu=1,
    resume_from_checkpoint=None, 
    val_check_interval = 0.5, 
    n_val=-1,
    n_train=-1,
    n_test=-1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)


args_dict.update({'output_dir': "./" + model_prefix + "_final", 'num_train_epochs':2,
             'train_batch_size': 1, 'eval_batch_size': 1})
args = argparse.Namespace(**args_dict)


## Define Checkpoint function
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath="./" + model_prefix + "_checkpoint", prefix=model_prefix, monitor="accuracy", mode="max", save_top_k=1
)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Checkpoint directory /content exists and is not empty.
  warnings.warn(*args, **kwargs)


In [ ]:
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    resume_from_checkpoint=args.resume_from_checkpoint,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    val_check_interval=args.val_check_interval,
    callbacks=[LoggingCallback()],
    # progress_bar_refresh_rate=0
)

model = LonformerFineTuner(args)
trainer = pl.Trainer(**train_params)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

In [ ]:
trainer.fit(model)


  | Name  | Type                                | Params
--------------------------------------------------------------
0 | model | LongformerForSequenceClassification | 148 M 
--------------------------------------------------------------
148 M     Trainable params
0         Non-trainable params
148 M     Total params


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch m

1

### Evaluate Model

In [ ]:
import textwrap
from tqdm.auto import tqdm
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from datasets import load_metric
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from transformers import (
    AdamW,
    LongformerTokenizer,
    LongformerForSequenceClassification,
    get_linear_schedule_with_warmup
)
import logging
import argparse
import time

In [ ]:
model_path = 'longformer-4096_final/longformer-4096'
tokenizer = LongformerTokenizer.from_pretrained(model_path)
model = LongformerForSequenceClassification.from_pretrained(model_path, return_dict=True, num_labels=2)

In [ ]:
import torch
train_dataset = torch.load('train_data.pt')
val_dataset = torch.load('valid_data.pt')
loader = DataLoader(val_dataset, batch_size=1, shuffle=True)
it = iter(loader)

In [ ]:
batch = next(it)
batch["input_ids"].shape

/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:851: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


torch.Size([1, 2048])

In [ ]:
batch["input_ids"] = batch["input_ids"].to('cuda')
batch["attention_mask"] = batch["attention_mask"].to('cuda')
batch["targets"] = batch["targets"].to('cuda')
model.to('cuda')
outputs = model.forward(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )

labels = batch['targets']
logits = F.softmax(outputs['logits'], dim=1)
print(logits)
preds = torch.argmax(logits, 1)
preds = preds.cpu().data.numpy()
targets = labels.cpu().data.numpy()    

texts = [tokenizer.decode(ids) for ids in batch['input_ids']]

tensor([[9.9985e-01, 1.4761e-04]], device='cuda:0', grad_fn=<SoftmaxBackward>)


In [ ]:
for i in range(1):
    lines = textwrap.wrap("Input Text:\n%s\n" % texts[i], width=100)
    print("\n".join(lines))
    print("\nActual Class: %s" % targets[i])
    print("\nPredicted Class from Longformer: %s" % preds[i])
    print("=====================================================================\n")

Input Text: <s>July 31 (UPI) &#8212; A tropical depression off the Gulf Coast of Florida was
upgraded to a tropical storm on Monday, the National Hurricane Center said. The storm was designated
Tropical Storm Emily on Monday morning &#8212; the fifth named storm of the 2017 Atlantic hurricane
season. Emily is moving eastward with strong winds at about 8 mph, and is expected to bring very
heavy rain to south and central Florida in the coming days. Monday, officials said 60 mph winds
forced the closure of the Sunshine Skyway Bridge in St. Petersburg and caused some infrastructure
damage. No serious injuries have yet been reported. A <a href="http://www.sun-
sentinel.com/news/weather/fl-reg-tropics-monday-20170731-story.html">tropical storm warning</a> was
issued early Monday for the west coast of Florida, from Anclote River southward to Bonita Beach. The
tropical storm made landfall at about 10:45 a.m., on Anna Maria Island, near Bradenton. Emily is
expected to turn northeast as it passe

In [ ]:
predictions = []
actual_labels = []

for batch in loader:
    batch["input_ids"] = batch["input_ids"].to('cuda')
    batch["attention_mask"] = batch["attention_mask"].to('cuda')
    batch["targets"] = batch["targets"]
    model.to('cuda')
    outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"]
            )

    labels = batch['targets']
    logits = F.softmax(outputs['logits'], dim=1)
    preds = torch.argmax(logits, 1)
    preds = preds.cpu().data.numpy()
    targets = labels.cpu().data.numpy()
    for i in range(len(batch)):
        predictions.append(preds)
        actual_labels.append(targets)


    do_print = False
    if do_print:  

        texts = [tokenizer.decode(ids) for ids in batch['source_ids']]
        
        lines = textwrap.wrap("Input Text:\n%s\n" % texts[i], width=100)
        print("\n".join(lines))
        print("\nActual Class: %s" % targets[i])
        print("\nPredicted Class from Longformer: %s" % preds[i])
        print("=====================================================================\n")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(actual_labels, predictions))

0.9336


In [ ]:
print(classification_report(actual_labels, predictions))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      7572
           1       0.92      0.95      0.93      7428

    accuracy                           0.93     15000
   macro avg       0.93      0.93      0.93     15000
weighted avg       0.93      0.93      0.93     15000

